# Edinburgh Medical School Demonstrates Competence as Medical Educator


![](https://www.studymedicineeurope.com/wp-content/uploads/2020/07/NP-_-General-Medicin-Council-7-2020-NS.jpg)

### Imports

In [1]:
from aries_cloudcontroller import AriesAgentController
import os
from termcolor import colored

### Initialise the Agent Controller

In [2]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://edinburgh-medical-school-agent:3021 and an api key of adminApiKey


### Start a Webhook Server

In [3]:
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

await agent_controller.init_webhook_server(webhook_host, webhook_port)

print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


## Register Agent Event Listeners

You can see some examples within the webhook_listeners recipe. Copy any relevant cells across and customise as needed.

In [4]:
listeners = []

In [5]:
## YOUR LISTENERS HERE
def holder_handler(payload):
    connection_id = payload['connection_id']
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    print("\n---------------------------------------------------\n")
    print("Handle Issue Credential Webhook")
    print(f"Connection ID : {connection_id}")
    print(f"Credential exchange ID : {exchange_id}")
    print("Agent Protocol Role : ", role)
    print("Protocol State : ", state )
    print("\n---------------------------------------------------\n")
    print("Handle Credential Webhook Payload")
    
    if state == "offer_received":
        print("Credential Offer Recieved")
        proposal = payload["credential_proposal_dict"]
        print("The proposal dictionary is likely how you would understand and display a credential offer in your application")
        print("\n", proposal)
        print("\n This includes the set of attributes you are being offered")
        attributes = proposal['credential_proposal']['attributes']
        print(attributes)
        ## YOUR LOGIC HERE
    elif state == "request_sent":
        print("\nA credential request object contains the commitment to the agents master secret using the nonce from the offer")
        ## YOUR LOGIC HERE
    elif state == "credential_received":
        print("Received Credential")
        ## YOUR LOGIC HERE
    elif state == "credential_acked":
        ## YOUR LOGIC HERE
        credential = payload["credential"]
        print("Credential Stored\n")
        print(credential)
        
        print("\nThe referent acts as the identifier for retrieving the raw credential from the wallet")
        # Note: You would probably save this in your application database
        credential_referent = credential["referent"]
        print("Referent", credential_referent)
    


    
holder_listener = {
    "topic": "issue_credential",
    "handler": holder_handler
}

listeners.append(holder_listener)

In [6]:
# Receive connection messages
def connections_handler(payload):
    state = payload['state']
    connection_id = payload["connection_id"]
    their_role = payload["their_role"]
    routing_state = payload["routing_state"]
    
    print("----------------------------------------------------------")
    print("Connection Webhook Event Received")
    print("Connection ID : ", connection_id)
    print("State : ", state)
    print("Routing State : ", routing_state)
    print("Their Role : ", their_role)
    print("----------------------------------------------------------")

    if state == "invitation":
        # Your business logic
        print("invitation")
    elif state == "request":
        # Your business logic
        print("request")

    elif state == "response":
        # Your business logic
        print("response")
    elif state == "active":
        # Your business logic
        print(colored("Connection ID: {0} is now active.".format(connection_id), "green", attrs=["bold"]))
        

connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

listeners.append(connection_listener)

In [7]:
agent_controller.register_listeners(listeners)

## GMC DID

Trusted DID throughout the Scottish Healthcare System

In [8]:
gmc_did = "TDAbSf3Uqebg8N4XvybMbg"

## Establish a Connection

Connection made with the GMC. This could be used in the future for secure communication between school and the GMC.

In [9]:
invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '193c2118-b8f3-4809-b2d9-e570e69a1347', 'recipientKeys': ['GzZxZieKMMF6jC3NCaVzPG7HtuLMWNNLatLpQN34ScwM'], 'serviceEndpoint': 'http://general-medical-council-agent:3020', 'label': 'General Medical Council'}


In [10]:
auto_accept=False
alias="General Medical School"

invite_response = await agent_controller.connections.receive_invitation(invitation, alias, auto_accept)
connection_id = invite_response["connection_id"]

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  101e39b1-45cf-4c61-b149-dfd2a08b9c58
State :  invitation
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
invitation
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  101e39b1-45cf-4c61-b149-dfd2a08b9c58
State :  request
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
request
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  101e39b1-45cf-4c61-b149-dfd2a08b9c58
State :  response
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
response
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  101e39b1-45cf-4c61-b149-dfd2a08b9c58
State :  active
Rout

In [ ]:
invite_response = await agent_controller.connections.accept_connection(connection_id)
connection_id = invite_response["connection_id"]

## Request Credential From Offer

Note: Your agent will automatically respond if ACAPY_AUTO_RESPOND_CREDENTIAL_OFFER=true flag is set in .env file of agent. Default is false.

To respond to an offer you must identify the offer using the credential_exchange_id generated for it. This is available from within the issue-credential holder handler. You could add custom logic in this loop `elif state == "request":` to handle this.

However, we will fetch the credential exchange records and **assume** this agent only has one record. Customise accordingly.


In [11]:
records_response = await agent_controller.issuer.get_records(connection_id=connection_id)
record = records_response["results"][0]
record_id = record["credential_exchange_id"]

In [12]:
await agent_controller.issuer.send_request_for_record(record_id)


---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : 101e39b1-45cf-4c61-b149-dfd2a08b9c58
Credential exchange ID : 72b39d4c-49bc-42d1-988f-27260aff46a4
Agent Protocol Role :  holder
Protocol State :  request_sent

---------------------------------------------------

Handle Credential Webhook Payload

A credential request object contains the commitment to the agents master secret using the nonce from the offer


{'credential_definition_id': 'TDAbSf3Uqebg8N4XvybMbg:3:CL:14:institution',
 'role': 'holder',
 'auto_offer': False,
 'auto_remove': False,
 'created_at': '2021-09-02 10:29:53.896988Z',
 'auto_issue': False,
 'trace': True,
 'initiator': 'external',
 'credential_request_metadata': {'master_secret_blinding_data': {'v_prime': '4991267454100260148575744900519863182280432681824417301361151630729203561794010025278436831336709982963082669293570671159860490611735039338170632627083924299255207297943824825197139048333758325214809783410663117085932851694187494076175018020644081762263528115943925502187003044029280823870058554581112705970036762305911757429385665982675736227726723279746330839132144998018121680215489799230227681357028762646969084503332633686177695554542400977864718230427758085006356784298528443889418918656123625762791840468905863001097678718953554963813676013611924777895747269159291001601107739359713993836473317565794941188655997369456005212403406176',
   'vr_prime': '222DCAD743FACE9


---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : 101e39b1-45cf-4c61-b149-dfd2a08b9c58
Credential exchange ID : 72b39d4c-49bc-42d1-988f-27260aff46a4
Agent Protocol Role :  holder
Protocol State :  credential_received

---------------------------------------------------

Handle Credential Webhook Payload
Received Credential


## Store Received Credential

This will be done automatically if the ACAPY_AUTO_STORE_CREDENTIAL=true flag is set in the .env file for this agent. Default is false.

Again you could handle this in your holder handler function in the `elif state == "credential_received":` loop.

In [13]:
# Optionally specify an identifier to uniquely identify this credential within your agents wallet.
# You would likely want to save this somewhere.
# If not set a random one will be generated for you
credential_id = "Medical School"

store_cred_response = await agent_controller.issuer.store_credential(record_id, credential_id)


---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : 101e39b1-45cf-4c61-b149-dfd2a08b9c58
Credential exchange ID : 72b39d4c-49bc-42d1-988f-27260aff46a4
Agent Protocol Role :  holder
Protocol State :  credential_acked

---------------------------------------------------

Handle Credential Webhook Payload
Credential Stored

{'referent': 'Medical School', 'attrs': {'Date Issued': '2021-09-02', 'Name': 'Edinburgh Medical School', 'Re-Validation Due': '2026-09-01'}, 'schema_id': 'TDAbSf3Uqebg8N4XvybMbg:2:Trusted School:0.0.6', 'cred_def_id': 'TDAbSf3Uqebg8N4XvybMbg:3:CL:14:institution', 'rev_reg_id': 'TDAbSf3Uqebg8N4XvybMbg:4:TDAbSf3Uqebg8N4XvybMbg:3:CL:14:institution:CL_ACCUM:7604c775-f123-49db-9675-c164d46e29fe', 'cred_rev_id': '1'}

The referent acts as the identifier for retrieving the raw credential from the wallet
Referent Medical School


## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [14]:
await agent_controller.terminate()